In [1]:
import tensorflow as tf

# reshape

In [4]:
a = tf.constant([[1.0, 2.0, 3.0, 4.0, 5.0, 6.0], [7.0, 8.0, 9.0, 10.0, 11.0, 12.0]])
print(a)

b = tf.reshape(a, (-1, 2, 3))
print(b)

tf.Tensor(
[[ 1.  2.  3.  4.  5.  6.]
 [ 7.  8.  9. 10. 11. 12.]], shape=(2, 6), dtype=float32)
tf.Tensor(
[[[ 1.  2.  3.]
  [ 4.  5.  6.]]

 [[ 7.  8.  9.]
  [10. 11. 12.]]], shape=(2, 2, 3), dtype=float32)


# Print

In [5]:
a = tf.constant([1.0])
b = tf.Print(a, [a], "a:") # No tf.Print in 2.0
print(b)

AttributeError: module 'tensorflow' has no attribute 'Print'

# custom gradients

In [14]:
@tf.custom_gradient
def replace_gradient(x, y):
    def grad(dy):
        return y, None
    return x, grad

a = tf.Variable([1.0, 2.0, 5.0], dtype=tf.float32)

with tf.GradientTape() as g:
    g.watch(a)
    c = tf.constant([3.0, 5.0, 9.0])

    b = replace_gradient(a, c)  
#     b = tf.identity(a)
    
gradients = g.gradient(b, a)
gradients

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3., 5., 9.], dtype=float32)>

In [41]:
@tf.custom_gradient
def replace_gradient(x, y):
    def grad(dy):
        return y, None
    return x, grad

a = tf.Variable([1.0, 2.0, 5.0], trainable=True, dtype=tf.float32)

with tf.GradientTape(persistent=True) as g:
    g.watch(a)
    c = tf.constant([3.0, 5.0, 9.0])

    b = replace_gradient(a, c) 
    
    loss = tf.reduce_sum(b)
    real_loss = 0 * loss
    
g_db_da = g.gradient(b, a)
g_dloss_da = g.gradient(loss, a)
g_drloss_da = g.gradient(real_loss, a)
print("db/da:", g_db_da)
print("loss:", loss)
print("dloss/da:", g_dloss_da)
print("drloss/da:", g_drloss_da)
print("a:", a)

optimizer = tf.keras.optimizers.SGD(learning_rate=-1)

optimizer.apply_gradients(zip([g_drloss_da], [a]))

print("updated a:", a)


# m = tf.Module()
# m.v = tf.Variable([1.])
# print(m.trainable_variables)

db/da: tf.Tensor([3. 5. 9.], shape=(3,), dtype=float32)
loss: tf.Tensor(8.0, shape=(), dtype=float32)
dloss/da: tf.Tensor([3. 5. 9.], shape=(3,), dtype=float32)
drloss/da: tf.Tensor([3. 5. 9.], shape=(3,), dtype=float32)
a: <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([1., 2., 5.], dtype=float32)>
updated a: <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([ 4.,  7., 14.], dtype=float32)>
